# 1. 데이터 받아 오기

# 2. 데이터 읽어오기

In [1]:
import glob
import os, re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


# 3. 데이터 다듬기

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 15: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence) # 공백을 제외하고 원하는 문장들만 출력! 결과를 보니 제대로 나온듯!!

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof
You saw her bathing on the roof
Her beauty and the moonlight overthrew her
She tied you
To a kitchen chair
She broke your throne, and she cut your hair
And from your lips she drew the Hallelujah Hallelujah


# 4. 전처리(필터링)

In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


# 5. 정재된 문장 모아보기

In [4]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

# 6. 토큰화

In [5]:
def tokenize(corpus):
    # 20000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 20000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=20000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f7ad3f8cc70>


### tensor 확인하기

In [6]:
print(tensor[:3, :15]) 

[[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
     0]
 [   2   17 2639  873    4    8   11 6043    6  329    3    0    0    0
     0]
 [   2   36    7   37   15  164  282   28  299    4   47    7   43    3
     0]]


### tokenizer에 구축된 단어 사전의 인덱스 확인하기

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 20: break 

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t
16 : s
17 : that
18 : on
19 : of
20 : .


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
enc = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
dec = tensor[:, 1:]    

print(enc[0])
print(dec[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(enc,dec, test_size=0.2, random_state=42)

# 7. 학습시키기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
BUFFER_SIZE = len(enc)
BATCH_SIZE = 256
steps_per_epoch = len(enc) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 20000개와, 여기 포함되지 않은 0:<pad>를 포함하여 20001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc, dec))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for enc_sample, dec_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(enc_sample)

<tf.Tensor: shape=(256, 14, 20001), dtype=float32, numpy=
array([[[ 6.45486070e-05, -2.69638786e-05,  1.68674509e-04, ...,
          1.05245272e-04,  1.15210692e-04,  3.25496476e-05],
        [-7.81954950e-05, -4.00930468e-04,  3.13864235e-04, ...,
         -1.60109150e-04,  1.59622417e-04,  1.02903568e-05],
        [ 1.40118245e-05, -6.39950798e-04,  3.75862670e-04, ...,
         -4.49892366e-04,  3.50168411e-05,  7.62780401e-05],
        ...,
        [ 7.85014068e-04, -8.24523799e-04,  1.23788801e-03, ...,
          3.78918048e-04,  1.20759527e-04, -1.15630543e-03],
        [ 7.80287082e-04, -9.40360827e-04,  1.20687799e-03, ...,
          3.87069042e-04,  2.18413261e-04, -1.50831114e-03],
        [ 7.43602228e-04, -1.06843794e-03,  1.14153896e-03, ...,
          4.03142272e-04,  2.90829485e-04, -1.83811889e-03]],

       [[ 6.45486070e-05, -2.69638786e-05,  1.68674509e-04, ...,
          1.05245272e-04,  1.15210692e-04,  3.25496476e-05],
        [ 3.70621965e-05,  9.68101631e-06,  2

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  5120256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  20501025  
Total params: 39,260,961
Trainable params: 39,260,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 149s 212ms/step - loss: 3.7497
Epoch 2/10
686/686 [==============================] - 159s 232ms/step - loss: 3.2177
Epoch 3/10
686/686 [==============================] - 166s 242ms/step - loss: 3.0257
Epoch 4/10
686/686 [==============================] - 166s 242ms/step - loss: 2.8814
Epoch 5/10
686/686 [==============================] - 166s 242ms/step - loss: 2.7564
Epoch 6/10
686/686 [==============================] - 167s 243ms/step - loss: 2.6445
Epoch 7/10
686/686 [==============================] - 166s 242ms/step - loss: 2.5417
Epoch 8/10
686/686 [==============================] - 166s 242ms/step - loss: 2.4458
Epoch 9/10
686/686 [==============================] - 167s 242ms/step - loss: 2.3569
Epoch 10/10
686/686 [==============================] - 167s 243ms/step - loss: 2.2728


In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

# 결과

In [22]:
generate_text(model, tokenizer, init_sentence="<start> she")

'<start> she s got a big ego hahaha <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> girl")

'<start> girl , i m a motherfucking monster <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> him")

'<start> him and the only one that s in the game <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> smoke")

'<start> smoke barney and <unk> , <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> voodoo")

'<start> voodoo davey with a <unk> tatted <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> sungjun")

'<start> <unk> <end> '

# 회고

### CV만 하다 첫 NLP 였는데 작사에 대한 학습 시키는 것이 생각 보다 재밌었습니다. 마지막 결과에서 제 이름으로 문장을 시작하려 했으나 < unk > 가 떴습니다. 학습한 데이터 중에 sunjun이라는 단어가 없어서 그런 것 같습니다. 